In [17]:
import cv2
import base64
import ollama

# 이미지 경로 설정
image_path = "test/Suwon_CH01_20200721_1500_TUE_9m_NH_highway_TW5_sunny_FHD_013.png"

# 이미지를 읽어서 Base64로 인코딩
with open(image_path, "rb") as img_file:
    encoded_image = base64.b64encode(img_file.read()).decode('utf-8')

# Ollama 모델에 요청
res = ollama.chat(
    model="llama3.2-vision:11b",
    messages=[
        {
            'role': 'user',
            'content': "tell me what is in this image. Answer only in [SEDAN/SUV/VAN/TRUCK/BUS]",
            'images': [encoded_image]
        }
    ]
)

# 응답 출력
print(res['message']['content'])


Sedan, SUV, Truck


In [ ]:
import ollama

res = ollama.chat(
	model= "llama3.2-vision:11b",
	messages=[
		{
			'role': 'user',
			'content': "tell me what is in this image. Answer only in [SEDAN/SUV/VAN/TRUCK/BUS]",
			'images': ['test/Suwon_CH01_20200721_1500_TUE_9m_NH_highway_TW5_sunny_FHD_013.png']
		}
	]
)

print(res['message']['content'])

In [1]:
import ollama

res = ollama.chat(
	model= "llama3.2-vision:11b",
	messages=[
		{
			'role': 'user',
			'content': "tell me what is in this image. Answer only in [SEDAN/SUV/VAN/TRUCK/BUS]",
			'images': ['test/Suwon_CH01_20200721_1500_TUE_9m_NH_highway_TW5_sunny_FHD_013.png']
		}
	]
)

print(res['message']['content'])

SUVs, sedans, vans, trucks and a bus.


In [16]:
%%time
res = ollama.chat(
	model= "llama3.2-vision:11b",
	messages=[
		{
			'role': 'user',
			'content': "tell me what is in this image. Answer only one in [SEDAN/SUV/VAN/TRUCK/BUS] answer in one word",
			'images': ['test/people_street.jpeg']
		}
	]
)

print(res['message']['content'])

Van.
CPU times: user 2.73 ms, sys: 0 ns, total: 2.73 ms
Wall time: 120 ms


In [18]:
from ultralytics import YOLO
import os
import xml.etree.ElementTree as ET
import random
import shutil
import yaml
import torch
import datetime
import cv2
import numpy as np
import os
import cv2
import csv
import numpy as np
from ultralytics import YOLO
import ollama
curr_dir = os.getcwd()

In [26]:
# 모델 로드
model = YOLO("yolo11x.pt").to("cuda") 
print(type(model.names),len(model.names))
print(f"Using device: {model.device}")
# 비디오 경로 설정
video_path = "test/traffic.mp4"
status_csv = "runs/status.csv"

<class 'dict'> 80
Using device: cuda:0


In [27]:
# car_crop 이미지를 Base64로 인코딩하는 함수
def encode_image_to_base64(image):
    _, buffer = cv2.imencode('.png', image)  # 이미지를 PNG 형식으로 인코딩
    encoded_image = base64.b64encode(buffer).decode('utf-8')  # Base64로 변환
    return encoded_image

In [28]:
## Tracking 없이 차종 복잡하게!!
crop_save_dir = os.path.join(curr_dir, "crops")
output_path = "runs/traffic_with_llm_fast.mp4"
# 비디오 읽기
cap = cv2.VideoCapture(video_path)
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = int(cap.get(cv2.CAP_PROP_FPS))
fourcc = cv2.VideoWriter_fourcc(*"mp4v")

# 비디오 저장 설정
out = cv2.VideoWriter(output_path, fourcc, fps, (width, height))

# 추적기 초기화
tracker = cv2.legacy.TrackerCSRT_create()  # CSRT 추적기 사용
trackers = cv2.legacy.MultiTracker_create()  # 멀티 추적기

frame_index = 0

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    results = model.predict(source=frame, save=False, stream=True)

    # 탐지된 객체를 바운딩 박스로 표시 및 추적 초기화
    for result in results:
        for i, box in enumerate(result.boxes):
            # 바운딩 박스 좌표 가져오기
            x1, y1, x2, y2 = map(int, box.xyxy[0])
            cls = int(box.cls)
            original_class_name = result.names[cls]
            if original_class_name == "car":  # 'car' 클래스만 Ollama 호출
                # 'car' 영역을 잘라내기
                car_crop = frame[y1:y2, x1:x2]
                encoded_car_crop = encode_image_to_base64(car_crop)  # 이미지를 Base64로 인코딩
                # Ollama API 호출
                res = ollama.chat(
                    model="llama3.2-vision:11b",
                    messages=[
                        {
                            'role': 'user',
                            'content': "tell me what is in this image. Answer only one in [SEDAN/SUV/VAN/TRUCK/BUS] answer in one word",
                            'images': [encoded_car_crop]
                        }
                    ]
                )

                final_result = res['message']['content'].strip().upper().replace(".", "").replace(" ", "")
                print(f"| Ollama Result: {final_result}")
                original_class_name = final_result
                # 바운딩 박스 그리기
                bbox_color = (0, 255, 0)  # 초록색 바운딩 박스
                thickness = 2
                cv2.rectangle(frame, (x1, y1), (x2, y2), bbox_color, thickness)

                # 텍스트 추가
                font = cv2.FONT_HERSHEY_SIMPLEX
                font_scale = 0.6
                text_color = (0, 0, 255)  # 빨간색 텍스트
                text_position = (x1, y1 - 10)
                cv2.putText(frame, final_result, text_position, font, font_scale, text_color, thickness)
            
            current_time = datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
            with open(status_csv, mode="a", newline="") as file:
                writer = csv.writer(file)
                writer.writerow([current_time, original_class_name, os.path.basename(video_path)])

    # 결과 프레임 저장
    out.write(frame)
    frame_index += 1

# 비디오 처리 종료
cap.release()
out.release()
cv2.destroyAllWindows()

print(f"Processed video saved at: {output_path}")


0: 384x640 15 cars, 22.9ms
| Ollama Result: SEDAN
| Ollama Result: SEDAN


KeyboardInterrupt: 